# Description

RAG baseline for kodeks chat, based on: 

1. https://habr.com/ru/articles/729664/
2. https://python.langchain.com/docs/use_cases/question_answering/quickstart/

Vectorizer: https://huggingface.co/cointegrated/rubert-tiny2

LLM: https://huggingface.co/IlyaGusev/saiga2_13b_gguf

# Installing Dependencies

In [1]:
!pip install langchain pandas tiktoken huggingface_hub
!pip install langchain_community
!pip install sentence-transformers
!pip install faiss-gpu
!pip install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 42.4 MB/s eta 0:00:0000:0100:01


In [6]:
!pip install numpy==1.24.4

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [4]:
!CMAKE_ARGS="-DLLAMA_CUDA=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 78.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 81.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 257.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 KB 150.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp310-cp310-linux_x86_64.whl size=56040165 sha256=70c6c96ef3c7f73d08543bf2682dcecb8671eb3335322c1dfe273c9f47cb2902
  Stored in directory: /tmp/pip-ephem-wheel-cache-hocw687e/wheels/e4/18/46/58b5c613b17c8d000d79ae650980fe871b3b490e04e6faa1c1
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensio

# Documents

In [2]:
# Несколько фраз, взятых из файла frag.pikcle
documents = [
    {'id': '1000001035', 'paragraph_number': 67, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.20. В п.6.2 паспорта количество населения, проживающего в зонах наблюдения, а также значения коллективной дозы облучения населения за счет деятельности организаций в целом по территории определяются путем суммирования соответствующих величин, взятых из пп.3.2 и 3.3 радиационно-гигиенических паспортов всех организаций, расположенных на территории (за исключением доз облучения пациентов медучреждений). Значение средней эффективной дозы облучения населения, проживающего в зонах наблюдения организаций, находящихся на данной территории, определяется путем деления полученного значения годовой коллективной дозы на общую численность населения, проживающего в зонах наблюдения организаций.'},
    {'id': '1000001035', 'paragraph_number': 68, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.21. В п.6.3 паспорта заносятся данные о структуре годовой коллективной дозы населения, проживающего на территории, за счет всех основных видов облучения.'},
    {'id': '1000001035', 'paragraph_number': 69, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.22. Годовая коллективная доза населения территории за счет деятельности организаций, использующих источники ионизирующего излучения (п.6.3.а паспорта территорий), определяется путем суммирования коллективных доз облучения персонала групп А и Б всех организаций, расположенных на территории, которые приведены в п. 3.3 паспортов организаций, и коллективной дозы облучения населения, проживающего в зонах наблюдения этих организаций по данным п.6.2 паспорта территории.'},
    {'id': '1000001035', 'paragraph_number': 70, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.23. В п.6.3.б паспорта территорий указывают коллективную эффективную дозу облучения всего населения территории в отчетном году, обусловленную загрязнением окружающей среды долгоживущими радионуклидами вследствие глобальных выпадений продуктов ядерных испытаний, прошлых радиационных аварий, и нормальной деятельности организаций (без радиационных аварий) за весь период, предшествующий отчетному году. Оценку дозы осуществляют территориальные ЦГСЭН с привлечением при необходимости научно-исследовательских учреждений системы Госсанэпидслужбы России согласно специальным методическим документам, а до их введения - согласно приложению 3 к методическим указаниям (далее по тексту - к МУ).'},
    {'id': '1000001035', 'paragraph_number': 71, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.24. Для оценки эффективной дозы облучения населения природными источниками ионизирующего излучения (п.6.3.в паспорта территорий) необходимо оценить дозы от всех ее составляющих: дозы космического излучения, дозы гамма-излучения земных пород и строительных конструкций, дозы внутреннего облучения за счет поступления природных радионуклидов с продуктами питания и водой, дозы за счет ингаляции изотопов радона, торона и их короткоживущих дочерних продуктов. Основной вклад в эффективную дозу облучения населения природными источниками вносит радон и его короткоживущие продукты. Методика проведения такой оценки приведена в приложении 4 (к МУ).'},
    {'id': '1000001035', 'paragraph_number': 72, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.25. Годовая эффективная коллективная доза всего населения, проживающего на территории, от медицинских исследований (п.6.3.г паспорта территорий) рассчитывается путем суммирования коллективных доз от основных видов этих исследований (флюорографических, рентгенографических, рентгеноскопических, радионуклидных).'},
    {'id': '1000001035', 'paragraph_number': 73, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.26. В п.6.З.е паспорта территорий указывают годовую коллективную дозу облучения всего населения территории, обусловленную радиационными авариями, происшедшими в отчетном году. В случае таких радиационных аварий индивидуальные дозы жителей территории должны быть определены согласно специальным методическим документам, разработанным применительно к условиям конкретных аварий и утвержденным Минздравом России. Коллективную дозу облучения населения территории от радиационных аварий вычисляют как сумму индивидуальных доз у всех жителей, подвергшихся облучению.'},
    {'id': '1000001035', 'paragraph_number': 74, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.27. Для определения относительного вклада каждого из основных видов облучения населения в суммарную (общую) коллективную дозу от всех источников необходимо вначале подсчитать сумму коллективных доз от этих источников. Приняв полученное суммарное значение коллективной дозы облучения населения за 100%, следует определить долю, приходящуюся на каждый вид облучения населения. Полученные относительные значения (в %) записываются в пп.6.3.а - 6.З.е паспорта территорий в скобках после каждого из соответствующих абсолютных значений коллективной дозы в чел.-Зв от различных видов облучения.'},
    {'id': '1000001035', 'paragraph_number': 75, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.28. В п.7 паспорта количество радиационных аварий и происшествий (аварийных ситуаций) определяется путем суммирования данных по всем организациям территории, взятым из п.5 радиационно-гигиенических паспортов организаций, а также по тем авариям и происшествиям, которые имели место в отчетном году на территории, однако по каким-либо причинам не были включены в радиационно-гигиенические паспорта организаций, но вошли в учетные формы системы Госсанэпидслужбы России.'},
    {'id': '1000001035', 'paragraph_number': 76, 'text': '4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.29. В п. 8 паспорта указывается наличие и число случаев лучевой патологии (число впервые выявленных заболеваний в отчетном году), если таковые имели место. Заполняется на основании журналов учета лиц, у которых впервые обнаружено профессиональное отравление и профзаболевание, по заключениям территориального центра (отделения) профпатологии или иного уполномоченного учреждения. Случаи лучевой патологии, обусловленные лучевой терапией, в паспорт не включаются.'}
]

# Vectorization

In [5]:
import pandas as pd

from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Cоздаем из наших документов датафрейм
df = pd.DataFrame(documents)
df.head()

,id,paragraph_number,text
0,1000001035,67,4. Ведение радиационно-гигиенического паспорта...
1,1000001035,68,4. Ведение радиационно-гигиенического паспорта...
2,1000001035,69,4. Ведение радиационно-гигиенического паспорта...
3,1000001035,70,4. Ведение радиационно-гигиенического паспорта...
4,1000001035,71,4. Ведение радиационно-гигиенического паспорта...


In [6]:
# Грузим фрейм в лоадер, выделив колонку для векторизации
loader = DataFrameLoader(df, page_content_column='text')
documents = loader.load()

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Создаем сплиттер документов, чтобы уложиться в лимит по токенам
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Задаем векторайзер
model_name = 'cointegrated/rubert-tiny2'    # "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Создаем хранилище
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()

# Также можно сохранить хранилище локально
db.save_local('faiss_index')

# Тестируем ретривер
db.similarity_search_with_score('Как определяется годовая коллективная доза населения территории')

/home/dpkt/chernikov_workspace/kodekschat_baseline/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dpkt/chernikov_workspace/kodekschat_baseline/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/dpkt/chernikov_workspace/kodekschat_baseline/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[(Document(page_content='4. Ведение радиационно-гигиенического паспорта территории В целях повышения радиационной безопасности населения и реализации постановления Правительства Российской Федерации от 28.01.97 N 93 "О порядке разработки радиационно-гигиенических паспортов организаций и территорий"  приказываем:\n4.27. Для определения относительного вклада каждого из основных видов облучения населения в суммарную (общую) коллективную дозу от всех источников необходимо вначале подсчитать сумму коллективных доз от этих источников. Приняв полученное суммарное значение коллективной дозы облучения населения за 100%, следует определить долю, приходящуюся на каждый вид облучения населения. Полученные относительные значения (в %) записываются в пп.6.3.а - 6.З.е паспорта территорий в скобках после каждого из соответствующих абсолютных значений коллективной дозы в чел.-Зв от различных видов облучения.', metadata={'id': '1000001035', 'paragraph_number': 74}),
  0.79190993),
 (Document(page_conten

# LLM

In [8]:
# Загрузка модели и примера кода для взаимодействия
!wget https://huggingface.co/IlyaGusev/saiga2_13b_gguf/resolve/main/model-q4_K.gguf
!wget https://raw.githubusercontent.com/IlyaGusev/rulm/master/self_instruct/src/interact_llamacpp.py

--2024-05-06 15:12:02--  https://huggingface.co/IlyaGusev/saiga2_13b_gguf/resolve/main/model-q4_K.gguf
Resolving huggingface.co (huggingface.co)... 52.85.243.103, 52.85.243.72, 52.85.243.108, ...
Connecting to huggingface.co (huggingface.co)|52.85.243.103|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


302 Found
Location: https://cdn-lfs.huggingface.co/repos/ab/94/ab947ab33d7e2e2456ff30395e2002ba0873cd3831ebb1746d8b54903f298d86/c766b7a0f919d0bb06b3532329baf176b21d6edf76aed4ca09cebe104ca5b494?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-q4_K.gguf%3B+filename%3D%22model-q4_K.gguf%22%3B&Expires=1715256722&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTI1NjcyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9hYi85NC9hYjk0N2FiMzNkN2UyZTI0NTZmZjMwMzk1ZTIwMDJiYTA4NzNjZDM4MzFlYmIxNzQ2ZDhiNTQ5MDNmMjk4ZDg2L2M3NjZiN2EwZjkxOWQwYmIwNmIzNTMyMzI5YmFmMTc2YjIxZDZlZGY3NmFlZDRjYTA5Y2ViZTEwNGNhNWI0OTQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=noUW-yRfzeyM8h4WboCr3MiU2%7Eb1NKkjufoSDDPhl961btSZe2jIevHQEHMSZxtGIu5L37zeeL-CDQSM1CcqTrjaaBltuFf9Miin6sRh-uL2mRCXTqS5RQ1aZiB%7E92nqQwiyz1BQRSEIQnBlD92CePQwS3UuoQYKtgGjM404tFGE9VnCMSn0td2rf1Av-zWn0OqnNt5ArVylIG2f1F8040xYx8%7EMYZivK9GjE42rSPNBxtpDG3IKGmlTh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 1951 (1.9K) [text/plain]
Saving to: ‘interact_llamacpp.py.1’

interact_llamacpp.p 100%[===================>]   1.91K  --.-KB/s    in 0s      

2024-05-06 15:13:11 (28.0 MB/s) - ‘interact_llamacpp.py.1’ saved [1951/1951]



In [8]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

In [9]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [10]:
n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path='model-q4_K.gguf',# "/Users/rlm/Desktop/Code/llama.cpp/models/openorca-platypus2-13b.gguf.q4_0.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 16 key-value pairs and 363 tensors from model-q4_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 40
llama_model_load

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   no
ggml_cuda_init: CUDA_USE_TENSOR_CORES: yes
ggml_cuda_init: found 2 CUDA devices:
  Device 0: NVIDIA A100-PCIE-40GB, compute capability 8.0, VMM: yes
  Device 1: NVIDIA GeForce GT 1030, compute capability 6.1, VMM: yes
llm_load_tensors: ggml ctx size =    0.55 MiB
llm_load_tensors: offloading 40 repeating layers to GPU
llm_load_tensors: offloading non-repeating layers to GPU
llm_load_tensors: offloaded 41/41 layers to GPU
llm_load_tensors:        CPU buffer size =    87.89 MiB
llm_load_tensors:      CUDA0 buffer size =  7284.77 MiB
llm_load_tensors:      CUDA1 buffer size =   128.19 MiB
...................................................................................................
llama_new_context_with_model: n_ctx      = 512
llama_new_context_with_model: n_batch    = 512
llama_new_context_with_model: n_ubatch   = 512
llama_new_context_with_model: flash_attn = 0
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model

# Chain

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


prompt_template = """
Вы являетесь помощником при выполнении заданий по поиску ответов на вопросы.
Используйте приведенные ниже фрагменты из извлеченного контекста, чтобы ответить на вопрос.
Если вы не знаете ответа, просто скажите, что вы не знаете.
Используйте максимум три предложения и старайтесь, чтобы ответ был кратким.

Вопрос: {question}
Контекст: {context}
Ответ:
"""

prompt = PromptTemplate(
template=prompt_template, input_variables=['question', 'context']
)

In [15]:
# request = 'не знаю как связаться в поддержкой'
request = 'Как определяется годовая коллективная доза населения территории'
relevants = db.similarity_search(request)
doc = relevants[0].dict()['metadata']
doc['context'] = relevants[0].page_content.split('\n')[1]
doc['question'] = request
print(doc)

{'id': '1000001035', 'paragraph_number': 74, 'context': '4.27. Для определения относительного вклада каждого из основных видов облучения населения в суммарную (общую) коллективную дозу от всех источников необходимо вначале подсчитать сумму коллективных доз от этих источников. Приняв полученное суммарное значение коллективной дозы облучения населения за 100%, следует определить долю, приходящуюся на каждый вид облучения населения. Полученные относительные значения (в %) записываются в пп.6.3.а - 6.З.е паспорта территорий в скобках после каждого из соответствующих абсолютных значений коллективной дозы в чел.-Зв от различных видов облучения.', 'question': 'Как определяется годовая коллективная доза населения территории'}


In [16]:
chain = prompt | llm
chain.invoke(doc)

- Определение годовой коллективной дозы населения территории происходит путем подсчета суммарных значений коллективной дозы от всех источников облучения и расчета относительных долей каждого вида облучения на общее количество.


llama_print_timings:        load time =     543.09 ms
llama_print_timings:      sample time =       9.11 ms /    66 runs   (    0.14 ms per token,  7243.20 tokens per second)
llama_print_timings: prompt eval time =     542.75 ms /   356 tokens (    1.52 ms per token,   655.91 tokens per second)
llama_print_timings:        eval time =    1178.23 ms /    65 runs   (   18.13 ms per token,    55.17 tokens per second)
llama_print_timings:       total time =    1811.25 ms /   421 tokens


'- Определение годовой коллективной дозы населения территории происходит путем подсчета суммарных значений коллективной дозы от всех источников облучения и расчета относительных долей каждого вида облучения на общее количество.'

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Из выдачи ретривера брать только самый релевантный и только вторую строку
# (Временное решение для тестирования, которое можно изменить)
def format_docs(docs):
    return docs[0].page_content.split('\n')[1]


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

request = 'Как определяется годовая коллективная доза населения территории'
response = rag_chain.invoke(request)

Для определения годи

чной колле

Llama.generate: prefix-match hit


ктивной дозы населения территории необходимо подсчитать суммарную (общую) коллективную дозу от всех источников и затем распределить ее между основными видами облучения населения в соответствии с их относительным вкладом. Результат записывается в паспорте территорий.


llama_print_timings:        load time =     582.50 ms
llama_print_timings:      sample time =      11.65 ms /    84 runs   (    0.14 ms per token,  7210.30 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1587.53 ms /    84 runs   (   18.90 ms per token,    52.91 tokens per second)
llama_print_timings:       total time =    1694.74 ms /    85 tokens


In [25]:
print(f"Вопрос: {request.strip()}")
print(f"Ответ: {response.strip()}")

Вопрос: Как определяется годовая коллективная доза населения территории
Ответ: Для определения годичной коллективной дозы населения территории необходимо подсчитать суммарную (общую) коллективную дозу от всех источников и затем распределить ее между основными видами облучения населения в соответствии с их относительным вкладом. Результат записывается в паспорте территорий.
